## <font color= orange>**Global, Run & Agent Level Config**</font>

**The Agents SDK version (as of 0.2.9, released August 22, 2025) hasn't yet been updated to support the openai SDK changes that removed ResponseTextConfigParam. This mismatch causes the ImportError**

**Debug information**
* Agents SDK version: 0.2.8
* Python version: 3.12.2

```!pip install openai==0.28.0 ```

In [ ]:
!pip install openai==0.28.0


  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.101.0
    Uninstalling openai-1.101.0:
      Successfully uninstalled openai-1.101.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openai-agents 0.2.9 requires openai<2,>=1.99.6, but you have openai 0.28.0 which is incompatible.


In [ ]:
!pip install -Uq openai-agents

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

### **Global Config**

* Applies to the **entire SDK session or application**.
* Think of it as “default environment settings” for everything.
* Example: API keys, default LLM provider, logging, safety filters.

💡 Analogy: **School rules** → apply to all students unless a teacher overrides them.

In [ ]:
from agents import (
    Agent,
    Runner,
    AsyncOpenAI,
    OpenAIChatCompletionsModel,
    RunConfig,
    set_default_openai_client,
    set_default_openai_api,
    set_tracing_disabled
)
from google.colab import userdata
import os

# Global Configurations (apply to ALL agents/runs unless overridden)

set_default_openai_api("chat_completions")   # Global default: use chat-completions API
set_tracing_disabled(True)                   # Global default: disable tracing/logging everywhere

# API Key setup
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
os.environ["OPENAI_API_KEY"] = GEMINI_API_KEY   # Optional, for compatibility with OpenAI clients

MODEL_NAME = "gemini-2.5-flash"

# Global client (all agents/runs use this unless overridden)
external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)
set_default_openai_client(external_client)   # ✅ Global client override



# Agent-Level Configuration (specific agent instructions + model)

model = OpenAIChatCompletionsModel(
    model=MODEL_NAME,
    openai_client=external_client,   # agent uses the Gemini-backed client
)

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant",  # agent persona
    model=model,                                 # agent’s default model
)

# Run-Level Configuration (optional overrides for this run)

config = RunConfig(
    model=model,                 # Run-specific model (can override global/agent)
    model_provider=external_client,
    tracing_disabled=True,       # Run-specific tracing config (overrides global if needed)
)

# Execute the run with agent + input
result = Runner.run_sync(
    starting_agent=agent,
    input="Hello",
    run_config=config            # explicitly pass run config (best practice)
)

print(result.final_output)


Hello there! How can I help you today?



### **Agent Config**

* Applies to a **specific agent instance**.
* Defines the agent’s **role, tools, memory, and default model**.
* Example: FinanceAgent always uses `gpt-4` + stock API; TravelAgent uses `gpt-4-mini` + booking API.

💡 Analogy: **Each teacher’s teaching style** → math teacher vs history teacher.

In [ ]:
import asyncio
from openai import AsyncOpenAI
from agents import Agent, OpenAIChatCompletionsModel, Runner

# Client Setup (Gemini provider via OpenAI-compatible API)

client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)



# Agent-Level Configuration
# Here, the MODEL is bound directly to the agent itself.
# This means: whenever this agent is used, it will always call Gemini 2.5 Flash,
# regardless of any run config (unless overridden explicitly at run time).
async def main():
    agent = Agent(
        name="Assistant",
        instructions="You only respond in Urdu.",   # Agent persona
        model=OpenAIChatCompletionsModel(
            model="gemini-2.5-flash",               # Specific model for THIS agent
            openai_client=client,                   # Custom provider (Gemini API)
        ),
    )


    # Run Execution

    result = await Runner.run(
        agent,
        "I am learning Agentic AI with Panaversity Community",
    )

    # Output will be in Urdu because of the agent's instructions
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())


بہت خوب! پیناورسٹی کمیونٹی کے ساتھ ایجنٹک اے آئی سیکھنا ایک بہترین قدم ہے۔

آپ کی سیکھنے کی یہ کوشش بہت کامیاب ہو۔


### **Run Config**

* Applies to **a single agent run (conversation or task execution)**.
* Lets you override some global defaults for that run.
* Example: Change model or temperature **just for one run** without affecting others.

💡 Analogy: **Class session rules** → teacher says “today we do group work” even though school normally does lectures.




In [ ]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel
from agents.run import RunConfig


# LLM Client Setup (acts like a global but scoped manually here)

external_client = AsyncOpenAI(
    api_key=GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
)


# Model Configuration (used by this agent/run)

model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client,   # tells the model to use Gemini instead of OpenAI
)


# Agent-Level Configuration

agent = Agent(
    name="Assistant",
    instructions="You are a helpful assistant"
)

# Run-Level Configuration

config = RunConfig(
    model=model,                     # run will use this Gemini model
    model_provider=external_client,  # Gemini client for this run
    tracing_disabled=True            # disable tracing for this run
)

# Run Execution

result = Runner.run_sync(
    starting_agent=agent,
    input="Hello, how are you?",
    run_config=config                # RunConfig overrides what the agent uses
)

print(result.final_output)


I am doing well, thank you for asking! How are you today?

